# Force Atlas 2
# Skip notebook test

**Author: Hugo Linsenmaier**
    
In this notebook, we will see how large graph visualization can be achieved with cuGraph. Our runtime will be compared against a CPU based implementation targeting Networkx users.
    
RAPIDS Versions: 0.17
    
Test Hardware

GV100 32G, CUDA 11.0

# Introduction:


Force Atlas 2 is a force directed layout algorithm where nodes behave as particules and edges as springs. An iterative process will compute attractive and repulsive forces between these entities to converge in an equilibrium state where the drawing is visually interpretable by the user.


See https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0098679 for more details.


Please refer to the documentation https://docs.rapids.ai/api/cugraph/stable/api.html#module-cugraph.layout.force_atlas2 on how to use the different parameters.

In [ ]:
# The notebook compares cuGraph to Python fa2.
# therefore there some additional non-RAPIDS python libraries need to be installed. 
# Please run this cell if you need the additional libraries
! pip install fa2

In [2]:
# Import RAPIDS libraries

import cudf
import cugraph

In [3]:
# NetworkX and CPU based libraries

import networkx as nx
import pandas as pd
from fa2 import ForceAtlas2
import time

In [4]:
# Viz libraries

from cuxfilter.charts.datashader.custom_extensions.graph_assets import calc_connected_edges

import holoviews as hv

from colorcet import fire
from datashader.bundling import directly_connect_edges, hammer_bundle

from holoviews.operation.datashader import datashade, dynspread
from holoviews.operation import decimate

from dask.distributed import Client

In [5]:
# Define the parameters 
ITERATIONS=500
THETA=1.0
OPTIMIZE=True

In [6]:
# Define the path to the test data  
datafile = '../data/netscience.csv'

In [ ]:
# Setup Viz
client = Client()
hv.notebook_extension('bokeh','matplotlib')
decimate.max_samples=20000
dynspread.threshold=0.01
datashade.cmap=fire[40:]
sz = dict(width=150,height=150)
%opts RGB [xaxis=None yaxis=None show_grid=False bgcolor="black"]

# cuGraph

In [8]:
edges_gdf = cudf.read_csv(datafile, names=["source", "destination", "weights"],
                          delimiter=' ', dtype=["int32", "int32", "float32"])

In [9]:
G = cugraph.Graph()
G.from_cudf_edgelist(edges_gdf, renumber=False)
G.number_of_nodes(), G.number_of_edges()

(1589, 2742)

### Force Atlas 2 call

In [10]:
start = time.time()
pos_gdf = cugraph.layout.force_atlas2(G,
                                  max_iter=ITERATIONS,
                                  pos_list=None,
                                  outbound_attraction_distribution=True,
                                  lin_log_mode=False,
                                  edge_weight_influence=1.0,
                                  jitter_tolerance=1.0,
                                  barnes_hut_optimize=OPTIMIZE,
                                  barnes_hut_theta=THETA,
                                  scaling_ratio=2.0,
                                  strong_gravity_mode=False,
                                  gravity=1.0,
                                  verbose=False,
                                  callback=None)
elapsed = time.time() - start
print("Cugraph time : " + str(elapsed))

Cugraph time : 0.20984625816345215


###  Convert a graph into paths suitable for datashading

In [11]:
connected = calc_connected_edges(pos_gdf,
                                 edges_gdf,
                                 node_x="x",
                                 node_y="y",
                                 node_x_dtype="float32",
                                 node_y_dtype="float32",
                                 node_id="vertex",
                                 edge_source="source",
                                 edge_target="destination",
                                 edge_aggregate_col=None,
                                 edge_render_type="direct",
                                )

### Output

In [12]:
%%opts RGB [tools=["hover"] width=800 height=800]

r_direct = hv.Curve(connected, label="Direct")
datashade(r_direct)

:DynamicMap   []
   :RGB   [x,y]   (R,G,B,A)

# Python FA2

In [13]:
edges_df = pd.read_csv(datafile,
                       names=["source", "target", "weights"],
                       delimiter=' ',
                       dtype={"source" : "int32",
                              "target" : "int32",
                              "target" : "float32"
                              }
                      )

In [14]:
G_nx = nx.from_pandas_edgelist(edges_df, source='source', target='target')

### Force Atlas 2 call

In [15]:
start = time.time()
forceatlas2 = ForceAtlas2(outboundAttractionDistribution=True,  # Dissuade hubs
                          linLogMode=False,  # NOT IMPLEMENTED
                          adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                          edgeWeightInfluence=1.0,
                          
                          # Performance
                          jitterTolerance=1.0,  # Tolerance
                          barnesHutOptimize=OPTIMIZE,
                          barnesHutTheta=THETA,
                          multiThreaded=False,  # NOT IMPLEMENTED
                          
                          # Tuning
                          scalingRatio=2.0,
                          strongGravityMode=False,
                          gravity=1.0,
                          
                          # Log
                          verbose=False
                         )

pos_df = forceatlas2.forceatlas2_networkx_layout(G_nx,
                                                 iterations=ITERATIONS)
elapsed = time.time() - start
print("CPU time : " + str(elapsed))

CPU time : 5.9420082569122314


###    Convert a graph into paths suitable for datashading

In [16]:
pos_df = pd.DataFrame.from_dict(pos_df, orient='index', columns=['x', 'y'])
connected = directly_connect_edges(pos_df, edges_df)

### Output

In [17]:
%%opts RGB [tools=["hover"] width=800 height=800]

r_direct = hv.Curve(connected, label="Direct")
datashade(r_direct)

:DynamicMap   []
   :RGB   [x,y]   (R,G,B,A)

Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.